In [224]:
import sys
sys.path.append('../')
from datasets import load_dataset
from transformers import AutoTokenizer
import pandas as pd

model_checkpoint = "distilbert-base-cased"

tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)
datasets = load_dataset("squad")

Found cached dataset squad (/Users/prabhatkumar/.cache/huggingface/datasets/squad/plain_text/1.0.0/d6ec3ceb99ca480ce37cdd35555d6cb2511d223b9150cce08a837ef62ffea453)


  0%|          | 0/2 [00:00<?, ?it/s]

In [322]:
data_path = "/Users/prabhatkumar/Library/CloudStorage/GoogleDrive-888prabhat@gmail.com/My Drive/Work/nlp_transformer/data/jigsaw-toxic-comment-train.csv"
df = pd.read_csv(data_path)
def apply_finction(row):
    return [row['toxic'], row['severe_toxic'], row['obscene'], row['threat'], row['insult'], row['identity_hate']]

df['label'] = df.apply(apply_finction, axis=1) 
df['comment_text_1'] = df['comment_text']
df = df[['id', 'comment_text', 'comment_text_1','toxic', 'severe_toxic', 'obscene', 'threat', 'insult', 'identity_hate', 'label']]
df.to_csv('../data/train.csv', index=False)

In [323]:
df.head()

,id,comment_text,comment_text_1,toxic,severe_toxic,obscene,threat,insult,identity_hate,label
0,0000997932d777bf,Explanation\nWhy the edits made under my usern...,Explanation\nWhy the edits made under my usern...,0,0,0,0,0,0,"[0, 0, 0, 0, 0, 0]"
1,000103f0d9cfb60f,D'aww! He matches this background colour I'm s...,D'aww! He matches this background colour I'm s...,0,0,0,0,0,0,"[0, 0, 0, 0, 0, 0]"
2,000113f07ec002fd,"Hey man, I'm really not trying to edit war. It...","Hey man, I'm really not trying to edit war. It...",0,0,0,0,0,0,"[0, 0, 0, 0, 0, 0]"
3,0001b41b1c6bb37e,"""\nMore\nI can't make any real suggestions on ...","""\nMore\nI can't make any real suggestions on ...",0,0,0,0,0,0,"[0, 0, 0, 0, 0, 0]"
4,0001d958c54c6e35,"You, sir, are my hero. Any chance you remember...","You, sir, are my hero. Any chance you remember...",0,0,0,0,0,0,"[0, 0, 0, 0, 0, 0]"


In [324]:
from datasets import load_dataset
dataset = load_dataset("csv", data_files='../data/train.csv')

Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

Dataset csv downloaded and prepared to /Users/prabhatkumar/.cache/huggingface/datasets/csv/default-1df2ceda99e1ad7f/0.0.0/6954658bab30a358235fa864b05cf819af0e179325c740e4bc853bcc7ec513e1. Subsequent calls will reuse this data.


  0%|          | 0/1 [00:00<?, ?it/s]

In [325]:
dataset = dataset.filter(lambda example: len(example['comment_text']) > 4000)
dataset = dataset.filter(lambda example: len(example['comment_text_1']) > 4000)

dataset = dataset.filter(lambda example: example['comment_text'] != None)
dataset = dataset.filter(lambda example: example['comment_text_1'] != None)

Filter:   0%|          | 0/223549 [00:00<?, ? examples/s]

Filter:   0%|          | 0/1507 [00:00<?, ? examples/s]

Filter:   0%|          | 0/1507 [00:00<?, ? examples/s]

Filter:   0%|          | 0/1507 [00:00<?, ? examples/s]

In [326]:
dataset

DatasetDict({
    train: Dataset({
        features: ['id', 'comment_text', 'comment_text_1', 'toxic', 'severe_toxic', 'obscene', 'threat', 'insult', 'identity_hate', 'label'],
        num_rows: 1507
    })
})

In [328]:
import ast
label_cols = ['toxic', 'severe_toxic', 'obscene', 'threat', 'insult', 'identity_hate']

In [335]:
def prepare_train_features(examples):
    # Tokenize our examples with truncation and padding, but keep the overflows using a
    # stride. This results in one example possible giving several features when a context is long,
    # each of those features having a context that overlaps a bit the context of the previous
    # feature.
    #print(f"examples .... {examples}")
    #print(examples)
    examples["comment_text"] = [q.lstrip() for q in examples["comment_text"]]
    #print(f"comment_text    >>> {len(examples['comment_text'])} \n")
    examples["comment_text_1"] = [c.lstrip() for c in examples["comment_text_1"]]
    ans = []
    for val in examples["label"]:
        ans.append(ast.literal_eval(val))
    examples["label"] = ans
    #print(examples)
    #print(f"comment_text_1   >>> {len(examples['comment_text_1'])} \n")
    tokenized_examples = tokenizer(
        examples["comment_text"],
        examples["comment_text_1"],
        truncation="only_second",
        max_length=max_length,
        stride=doc_stride,
        return_overflowing_tokens=True,
        return_offsets_mapping=True,
        padding="max_length",
    )
   # print('..........')
    #print(f"tokenized_examples .... {tokenized_examples}")
    # Since one example might give us several features if it has a long context, we need a
    # map from a feature to its corresponding example. This key gives us just that.
    #overflow_to_sample_mapping = tokenized_examples.pop("overflow_to_sample_mapping")
    #sample_mapping = tokenized_examples["overflow_to_sample_mapping"]
    # The offset mappings will give us a map from token to character position in the original
    # context. This will help us compute the start_positions and end_positions.
    #offset_mapping = tokenized_examples.pop("offset_mapping")
    overflow_to_sample_mapping = tokenized_examples["overflow_to_sample_mapping"]
    offset_mapping = tokenized_examples["offset_mapping"]
    #print(offset_mapping[0])
    #print(f"OFFSET MAPPINg .... {len(offset_mapping)} \n")
    #print(f"OFFFSER MAPPING ...... {offset_mapping}")
    #print(sample_mapping)
    # Let's label those examples!
    #print(tokenized_examples)
    #ans=[]
    for label_col in label_cols:
        ans=[]
        for i, offsets in enumerate(overflow_to_sample_mapping):
            ans.append(examples[label_col][0])
        tokenized_examples[label_col] = ans
        #print(f"tokenized_examples['answers'] .... {tokenized_examples['answers']} \n")
    print(f'tokenized_examples .... {tokenized_examples} \n')
    return tokenized_examples

In [336]:
max_length = 2000  # The maximum length of a feature (question and context)
doc_stride = (
    128  # The authorized overlap between two part of the context when splitting
)
tokenized_datasets = dataset['train'].select(range(1)).map(
    prepare_train_features,
    batched=True,
    remove_columns=dataset["train"].column_names,
    num_proc=3,
    load_from_cache_file=False,
)

num_proc must be <= 1. Reducing num_proc to 1 for dataset of size 1.


Map:   0%|          | 0/1 [00:00<?, ? examples/s]

tokenized_examples .... {'input_ids': [[101, 7066, 1324, 117, 8667, 160, 8976, 12437, 20022, 119, 5556, 119, 5731, 119, 18563, 156, 17162, 15792, 1116, 131, 199, 197, 200, 225, 792, 793, 846, 844, 100, 794, 217, 108, 223, 842, 100, 222, 224, 100, 100, 100, 100, 855, 786, 789, 787, 790, 208, 221, 100, 100, 100, 100, 201, 100, 100, 109, 100, 100, 836, 100, 100, 341, 100, 100, 835, 100, 100, 100, 840, 100, 100, 202, 100, 100, 100, 100, 100, 100, 203, 876, 877, 878, 879, 6450, 14813, 23543, 1116, 131, 227, 247, 282, 283, 234, 255, 235, 259, 100, 100, 100, 307, 238, 265, 100, 100, 318, 319, 242, 272, 100, 275, 100, 335, 226, 246, 233, 254, 100, 258, 100, 264, 100, 271, 228, 248, 100, 100, 100, 256, 100, 100, 100, 100, 236, 260, 100, 100, 100, 266, 100, 100, 100, 273, 100, 333, 100, 334, 229, 250, 100, 257, 100, 261, 239, 268, 243, 274, 100, 277, 245, 100, 249, 100, 100, 100, 297, 237, 263, 100, 267, 100, 327, 100, 767, 232, 253, 100, 100, 100, 100, 100, 302, 100, 308, 100, 100, 320, 321, 32

In [321]:
#tokenized_datasets['input_ids'][0]

In [290]:
##d['comment_text']

num_proc must be <= 1. Reducing num_proc to 1 for dataset of size 1.


Map:   0%|          | 0/1 [00:00<?, ? examples/s]

ValueError: malformed node or string: ['[0, 0, 0, 0, 0, 0]']

In [99]:
question = "What was the World War II?"
ctx = """World War II, often abbreviated as WW2, was a global war that lasted from 1939 to 1945. It involved the vast majority of the world's countries forming two opposing military alliances: the Allies and the Axis powers."""

In [ ]:
    tokenized_examples = tokenizer(
        examples["question"],
        examples["context"],
        truncation="only_second",
        max_length=max_length,
        stride=doc_stride,
        return_overflowing_tokens=True,
        return_offsets_mapping=True,
        padding="max_length",
    )

In [155]:
question = ["What was the World War II?","Where was the World War II?"]
ctx = ["""World War II, often abbreviated as WW2, was a global war that lasted from 1939 to 1945. It involved the vast majority of the world's countries forming two opposing military alliances: the Allies and the Axis powers.""",
       """World War II, often abbreviated as WW2"""]

tokenized = tokenizer(question, ctx, truncation="only_second", stride=5, max_length=40, return_offsets_mapping=True,return_overflowing_tokens=True)
tokenized

{'input_ids': [[101, 1327, 1108, 1103, 1291, 1414, 1563, 136, 102, 1291, 1414, 1563, 117, 1510, 12258, 1112, 160, 2924, 1477, 117, 1108, 170, 4265, 1594, 1115, 5695, 1121, 3061, 1106, 2481, 119, 1135, 2017, 1103, 6047, 2656, 1104, 1103, 1362, 102], [101, 1327, 1108, 1103, 1291, 1414, 1563, 136, 102, 6047, 2656, 1104, 1103, 1362, 112, 188, 2182, 5071, 1160, 10137, 1764, 25012, 131, 1103, 14254, 1105, 1103, 17831, 3758, 119, 102], [101, 2777, 1108, 1103, 1291, 1414, 1563, 136, 102, 1291, 1414, 1563, 117, 1510, 12258, 1112, 160, 2924, 1477, 102]], 'attention_mask': [[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]], 'offset_mapping': [[(0, 0), (0, 4), (5, 8), (9, 12), (13, 18), (19, 22), (23, 25), (25, 26), (0, 0), (0, 5), (6, 9), (10, 12), (12, 13), (14, 19), (20

In [108]:
len(tokenized['input_ids'])

3

In [82]:
max_length = 384  # The maximum length of a feature (question and context)
doc_stride = (
    128  # The authorized overlap between two part of the context when splitting
)
tokenized_examples = tokenizer(
    even_datasets["validation"]["context"],                          
    even_datasets["validation"]["question"],
    truncation="only_second",
    max_length=max_length,
    stride=doc_stride,
    return_overflowing_tokens=True,
    return_offsets_mapping=True,
    padding="max_length",
)

Exception: Truncation error: Sequence to truncate too short to respect the provided max_length

In [54]:
even_datasets["train"]["question"]

['To whom did the Virgin Mary allegedly appear in 1858 in Lourdes France?',
 'When Konya Technical University opens, how many total institutes of technology will there be in Turkey and the Ottoman Empire?',
 'Which of the Marshall Islands did Salazar most likely see?',
 'To somewhat avoid the water vapor in the atmosphere, where can an observatory be sited?',
 'Who originally donated the land for Hospital Macie?',
 'Who decided in 1976 that Norfolk Island is a part of the Commonwealth?',
 'The northern and southern reaches of the desert are spare of what items?',
 'Which record labels really supported the British post-punk movement?',
 'What did Americo-liberians exclude tribes from?']

In [55]:
t = "The Americo-Liberian settlers did not identify with the indigenous peoples they encountered, especially those in communities of the more isolated bush. They knew nothing of their cultures, languages or animist religion. Encounters with tribal Africans in the bush often developed as violent confrontations. The colonial settlements were raided by the Kru and Grebo people from their inland chiefdoms"
t = t + t + t
q = "To whom did the Virgin Mary allegedly appear in 1858 in Lourdes France?"

In [32]:
max_length = 300  # The maximum length of a feature (question and context)
doc_stride = (
    50  # The authorized overlap between two part of the context when splitting
)
tokenized_examples = tokenizer(
    t,
    q,
    truncation="only_second",
    max_length=max_length,
    stride=doc_stride,
    return_overflowing_tokens=True,
    return_offsets_mapping=True,
    padding="max_length",
)

In [47]:
tokenizer.cls_token_id


101

In [50]:
#
# tokenized_examples['input_ids']

In [52]:
tokenized_examples['input_ids'][].index(tokenizer.cls_token_id)

IndexError: list index out of range

In [46]:
tokenized_examples

[[101,
  1109,
  7277,
  26237,
  1186,
  118,
  19427,
  1179,
  7056,
  1225,
  1136,
  6183,
  1114,
  1103,
  6854,
  7983,
  1152,
  8181,
  117,
  2108,
  1343,
  1107,
  3611,
  1104,
  1103,
  1167,
  6841,
  13771,
  119,
  1220,
  1450,
  1720,
  1104,
  1147,
  8708,
  117,
  3483,
  1137,
  1126,
  4060,
  1776,
  4483,
  119,
  13832,
  2528,
  8355,
  1468,
  1114,
  10059,
  19442,
  1107,
  1103,
  13771,
  1510,
  1872,
  1112,
  5973,
  14002,
  1116,
  119,
  1109,
  5929,
  7536,
  1127,
  20574,
  1118,
  1103,
  148,
  5082,
  1105,
  144,
  1874,
  4043,
  1234,
  1121,
  1147,
  11054,
  2705,
  9277,
  1116,
  1942,
  4638,
  7277,
  26237,
  1186,
  118,
  19427,
  1179,
  7056,
  1225,
  1136,
  6183,
  1114,
  1103,
  6854,
  7983,
  1152,
  8181,
  117,
  2108,
  1343,
  1107,
  3611,
  1104,
  1103,
  1167,
  6841,
  13771,
  119,
  1220,
  1450,
  1720,
  1104,
  1147,
  8708,
  117,
  3483,
  1137,
  1126,
  4060,
  1776,
  4483,
  119,
  13832,
  2528,


In [83]:
max_length = 384  # The maximum length of a feature (question and context)
doc_stride = (
    128  # The authorized overlap between two part of the context when splitting
)

In [11]:
tokenized_datasets = even_datasets.map(
    prepare_train_features,
    batched=True,
    remove_columns=even_datasets["train"].column_names,
    num_proc=3,
    load_from_cache_file=False,
)

Map (num_proc=3):   0%|          | 0/9 [00:00<?, ? examples/s]

4
43
4
3

[[(0, 0), (0, 2), (3, 7), (8, 11), (12, 15), (16, 22), (23, 27), (28, 37), (38, 44), (45, 47), (48, 52), (53, 55), (56, 59), (59, 63), (64, 70), (70, 71), (0, 0), (0, 13), (13, 15), (15, 16), (17, 20), (21, 27), (28, 31), (32, 33), (34, 42), (43, 52), (52, 53), (54, 56), (56, 58), (59, 62), (63, 67), (68, 76), (76, 77), (77, 78), (79, 83), (84, 88), (89, 91), (92, 93), (94, 100), (101, 107), (108, 110), (111, 114), (115, 121), (122, 126), (126, 127), (128, 139), (140, 142), (143, 148), (149, 151), (152, 155), (156, 160), (161, 169), (170, 173), (174, 180), (181, 183), (183, 184), (185, 187), (188, 189), (190, 196), (197, 203), (204, 206), (207, 213), (214, 218), (219, 223), (224, 226), (226, 229), (229, 232), (233, 237), (238, 241), (242, 248), (249, 250), (250, 251), (251, 254), (254, 256), (257, 259), (260, 262), (263, 264), (264, 265), (265, 268), (268, 269), (269, 270), (271, 275), (276, 278), (279, 282), (283, 287), (288, 296), (297, 299), (300, 303), (304, 312), (313, 3

num_proc must be <= 2. Reducing num_proc to 2 for dataset of size 2.


Map (num_proc=2):   0%|          | 0/2 [00:00<?, ? examples/s]

44

11

[[(0, 0), (0, 5), (6, 13), (14, 19), (20, 22), (23, 24), (25, 32), (33, 35), (36, 39), (40, 47), (48, 54), (54, 55), (0, 0), (0, 4), (5, 13), (14, 16), (17, 20), (21, 28), (29, 32), (33, 40), (41, 43), (44, 49), (50, 56), (57, 60), (61, 74), (75, 80), (80, 81), (82, 85), (86, 93), (94, 100), (101, 104), (105, 107), (108, 111), (112, 118), (119, 121), (122, 134), (135, 142), (143, 146), (147, 154), (155, 158), (159, 166), (167, 173), (174, 177), (178, 181), (182, 191), (191, 192), (193, 202), (203, 204), (205, 209), (210, 216), (217, 219), (220, 228), (229, 232), (232, 237), (238, 245), (246, 254), (255, 261), (261, 262), (263, 266), (267, 274), (275, 281), (282, 291), (292, 300), (301, 305), (306, 309), (310, 314), (315, 325), (326, 333), (333, 334), (335, 338), (339, 345), (346, 352), (353, 357), (358, 365), (366, 368), (369, 372), (373, 378), (379, 383), (384, 391), (391, 392), (393, 397), (398, 401), (402, 410), (411, 419), (420, 422), (423, 433), (434, 437), (438, 442), (44

In [7]:
print(datasets["train"][100])


{'id': '573387acd058e614000b5cb5', 'title': 'University_of_Notre_Dame', 'context': 'One of the main driving forces in the growth of the University was its football team, the Notre Dame Fighting Irish. Knute Rockne became head coach in 1918. Under Rockne, the Irish would post a record of 105 wins, 12 losses, and five ties. During his 13 years the Irish won three national championships, had five undefeated seasons, won the Rose Bowl in 1925, and produced players such as George Gipp and the "Four Horsemen". Knute Rockne has the highest winning percentage (.881) in NCAA Division I/FBS football history. Rockne\'s offenses employed the Notre Dame Box and his defenses ran a 7–2–2 scheme. The last game Rockne coached was on December 14, 1930 when he led a group of Notre Dame all-stars against the New York Giants in New York City.', 'question': 'In what year did the team lead by Knute Rockne win the Rose Bowl?', 'answers': {'text': ['1925'], 'answer_start': [354]}}


Map (num_proc=3):   0%|          | 0/87599 [00:00<?, ? examples/s]

Map (num_proc=3):   0%|          | 0/10570 [00:00<?, ? examples/s]

In [10]:
train_set = tokenized_datasets["train"].with_format("numpy")[:]


In [11]:
train_set

{'input_ids': array([[  101,  1706,  2292, ...,     0,     0,     0],
        [  101,  1327,  1110, ...,     0,     0,     0],
        [  101,  1109, 19349, ...,     0,     0,     0],
        ...,
        [  101,  1556,  1184, ...,     0,     0,     0],
        [  101,  1130,  1184, ...,     0,     0,     0],
        [  101,  1327,  1110, ...,     0,     0,     0]]),
 'attention_mask': array([[1, 1, 1, ..., 0, 0, 0],
        [1, 1, 1, ..., 0, 0, 0],
        [1, 1, 1, ..., 0, 0, 0],
        ...,
        [1, 1, 1, ..., 0, 0, 0],
        [1, 1, 1, ..., 0, 0, 0],
        [1, 1, 1, ..., 0, 0, 0]]),
 'start_positions': array([136,  53,  83, ..., 116,  52,  11]),
 'end_positions': array([142,  57,  85, ..., 116,  52,  15])}